# Part 1 - Get used to the OpenAI API
 

## Import

Import necessary libraries.

In [1]:
!pip install openai tiktoken sentence_transformers pandas numpy python-dotenv

In [2]:
import openai
from openai import AzureOpenAI
import os
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np
import tiktoken
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv  
load_dotenv();

/opt/conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Constants

Let's set up some constants. These are:

- The model we will be using (defined in Azure OpenAI).
- Cost for token completion and prompt (we take this from Azure OpenAI Studio for the respective model).
- Openai API setup.

In [3]:
# MODEL = os.environ.get("OPENAI_API_DEPLOYMENT", "gpt-4")
MODEL = "gpt-35-turbo"  # Uncomment if token rate is too high

%run model_cost.py
model_cost = get_model_instance(model_name=MODEL)

client = AzureOpenAI(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
)

In [4]:
print(os.environ.get("AZURE_OPENAI_API_KEY"))

3606312c6a244aa2a4ed105cba1c3e5e


## Construct a prompt and receive response from the API

We will send a prompt to the API and get a response back. The prompt is a string that is used to "seed" the model. The model will then generate a completion based on the prompt. The completion is a string of text that is generated by the model.

In [5]:
def ask_question(
        prompt: List[Dict[str, str]],
        model: str = MODEL
    ) -> openai.types.chat.chat_completion.ChatCompletion:
    """Function to ask a question to the GPT model using the Azure OpenAI API.
    
    Args:
        prompt: The prompt to send to the GPT model
        model: The model to use

    Returns:
        The response from the GPT model
    """

    response = client.chat.completions.create(
        model=model,
        messages=prompt,
        temperature=0.7,
        max_tokens=1500,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
    )
    
    return response


It is always a good idea to monitor costs. We will do this with the function `get_cost()`. This function will return the cost of the API call.

In [6]:
def get_cost(
        response: openai.types.chat.chat_completion.ChatCompletion,
        token_cost_per_completion: float,
        token_cost_per_prompt: float,
    ) -> float:
    """Function to compute the cost of a prompt + completion response.
    
    Args:
        response: The response from the GPT model
        token_cost_per_completion: The cost per completion token
        token_cost_per_prompt: The cost per prompt token
    
    Returns:
        The cost of the prompt + completion response
    """
    completion_tokens = response.usage.completion_tokens
    prompt_tokens = response.usage.prompt_tokens

    cost = (completion_tokens * token_cost_per_completion) + (
        prompt_tokens * token_cost_per_prompt
    )

    return cost


In [7]:
# Define a question to ask to the model
question = "Which one is the best Greek Island for a vacation?"

# Set the character
character = "You answer question about the Greek Islands. Include a joke about Greece in every response."

# Build the prompt
prompt = [
    {
        "role": "system",
        "content": character
    },
    {
        "role": "user",
        "content": question
    },
]

# Ask the question to the model
response = ask_question(prompt=prompt)

# Display the answer
answer = response.choices[0].message.content
display(Markdown(answer))

# Compute and print the cost
cost = get_cost(
    response=response,
    token_cost_per_completion=model_cost.token_cost_per_completion,
    token_cost_per_prompt=model_cost.token_cost_per_prompt
)
print(f"Total cost for this response: {cost:.5f} {model_cost.currency} (model was {model_cost.name}).")

It really depends on what you're looking for in a vacation. If you're into picturesque beaches and vibrant nightlife, Mykonos is a popular choice. If you prefer a more relaxed and traditional atmosphere, Santorini with its stunning sunsets and white-washed buildings is a great option. And if you're interested in history and culture, Crete, the largest of the Greek islands, offers a mix of beautiful landscapes and ancient ruins.

Joke: Why don't Greeks like making eye contact during a conversation? Because it's hard to focus with all that feta in their eyes!

Total cost for this response: 0.00028 CHF (model was gpt3.5-turbo).


In [8]:
response

ChatCompletion(id='chatcmpl-9UVIKiVtG91XmsP087dUkfTouMuI1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="It really depends on what you're looking for in a vacation. If you're into picturesque beaches and vibrant nightlife, Mykonos is a popular choice. If you prefer a more relaxed and traditional atmosphere, Santorini with its stunning sunsets and white-washed buildings is a great option. And if you're interested in history and culture, Crete, the largest of the Greek islands, offers a mix of beautiful landscapes and ancient ruins.\n\nJoke: Why don't Greeks like making eye contact during a conversation? Because it's hard to focus with all that feta in their eyes!", role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'seve

# Part 2 - Integrating Private Domain Knowledge

Note that you need to have executed all cells in Part 1 for Part 2 to function.

## High level workflow description

<img src="images/0_overview.png" alt="Image description" width="1000">  


## Load the Travel Articles and Calculate the Embeddings

We have prepared the travel articles for you. These articles constitute private domain knowledge that the model has no access to. We will load them into memory and calculate the embeddings for each article. We will use these embeddings to retrieve the most relevant articles for a given question.

<img src="images/1_calculate_domain_knowledge.png" alt="Image description" width="200">  


You need to provide the path to the travel articles to the Notebooks local instance and provide the path to the travel articles in the variable `path_travel_articles`.

In [9]:
# Provide the path to the embeddings file as a string, e.g., path_travel_articles = "./data/travel_articles/"
path_travel_articles = "./data/travel_articles/"

Each article is in a separate text file. The code snipped below will discover all the text files in the directory and load them into memory.

In [10]:
travel_articles = []
for file in os.listdir(path_travel_articles):
    if file.endswith(".txt"):
        with open(os.path.join(path_travel_articles, file), "r") as f:
            travel_articles.append(f.read())

display(Markdown(travel_articles[17]))

Title: Run Your Way Through Paradise: Evia, the Ultimate Running Destination

Subtitle: Discover the spectacular island of Evia in Greece and unleash your inner runner in the world's premier running paradise

Introduction

Trade in your treadmill for a breathtaking journey to the idyllic island of Evia, Greece's best-kept secret and the ultimate running destination. With its pristine coastline, lush forests, and majestic mountains, Evia has something for everyone, whether you're a seasoned marathoner or a casual jogger. In this article, we'll explore what makes Evia the perfect getaway for those seeking a running adventure like no other and highlight the must-visit attractions that make this island so special.

Running Paradise: What Makes Evia Stand Out

Evia, the second-largest island in Greece, boasts an unparalleled landscape that combines breathtaking beauty with a diverse terrain that caters to runners of all levels. Its 370 km of coastline offers stunning views, whether you're running along the sandy beaches of the Aegean Sea or traversing the rocky cliffs that frame the Euboean Gulf. The island's lush, green forests provide a peaceful escape from the bustling cities, while the challenging mountain trails will test your stamina and reward you with unparalleled vistas.

But what truly sets Evia apart is its unique "Runners' Village", an innovative concept that brings together running enthusiasts from around the world to share their passion, exchange tips, and embark on unforgettable running adventures. Located in the heart of the island, the Runners' Village offers state-of-the-art facilities, expert coaching, and a supportive community that will help you elevate your running game to new heights.

Runners' Village: A Haven for Running Enthusiasts

Established in 20XX, the Runners' Village has quickly become a mecca for running enthusiasts, attracting athletes from around the globe. The Village offers a range of accommodations to suit every budget, from cozy cottages to luxurious villas, all designed with the runner's comfort and needs in mind.

The heart of the Runners' Village is the state-of-the-art Training Center, equipped with the latest technology and staffed by a team of expert coaches who are on hand to help you achieve your running goals. Whether you're looking to improve your technique, increase your speed, or simply learn more about the sport, the Training Center offers a range of workshops, seminars, and one-on-one coaching sessions tailored to your needs.

But the true magic of the Runners' Village lies in its diverse running trails, which have been meticulously designed to cater to all abilities and preferences. From gentle beachside jogs to grueling mountain treks, the island's extensive trail network offers an unrivaled running experience. And with regular races and events, including the prestigious Evia Marathon, there's always an opportunity to test your skills and challenge yourself against runners from around the world.

Evia's Must-Visit Attractions

While running is undoubtedly the main attraction, Evia has much more to offer. Here are some must-visit attractions to make your stay on the island even more memorable:

1. Edipsos Hot Springs: After a long run, treat your tired muscles to a soothing soak in the ancient hot springs of Edipsos, famed for their therapeutic properties since the days of Aristotle.

2. Steni Forest: Discover the island's lush, green heart by taking a leisurely hike through Steni Forest, a tranquil haven home to a rich variety of flora and fauna, including the elusive roe deer.

3. Drakospita: Explore the mysterious Drakospita, or "Dragon Houses", ancient stone structures that have puzzled archaeologists for centuries. Whether you believe they were the homes of mythological creatures or simply impressive feats of engineering, they're a must-see.

4. Kymi: Visit the picturesque town of Kymi, perched on a hill overlooking the Aegean Sea. With its narrow, winding streets and traditional architecture, it's the perfect spot for a leisurely stroll and a taste of authentic Greek cuisine.

5. Lihadonisia: Take a day trip to the uninhabited Lihadonisia islands, known as the "Seychelles of Greece" for their crystal-clear waters and pristine beaches. Snorkel, swim, or simply soak up the sun in this idyllic paradise.

In Conclusion

Evia is a hidden gem that offers a running experience like no other. With its diverse landscape, state-of-the-art facilities, and welcoming community, it's no wonder that the island has become the ultimate destination for running enthusiasts. So lace up your running shoes, pack your bags, and prepare to discover the unparalleled beauty and exhilarating trails of Evia, the world's premier running paradise.

Next, we calculate the embeddings using the sentence transformer model `all-mpnet-base-v2` (https://huggingface.co/sentence-transformers/all-mpnet-base-v2). Observe that all the vectors are of the same size, `768`. This is because the transformer model outputs a fixed-size vector for each input string.

In [11]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")

embeddings = []
for article in travel_articles:
    embedding = embedding_model.encode(article, show_progress_bar=True)
    embeddings.append(embedding)
    print(f"Shape of the vector: {embedding.shape}")

embeddings[17]

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Shape of the vector: (768,)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Shape of the vector: (768,)


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Shape of the vector: (768,)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Shape of the vector: (768,)


array([-4.36107963e-02, -2.85593532e-02, -5.36731668e-02,  8.87268186e-02,
        4.92947921e-02,  2.16004699e-02, -1.24554038e-02,  3.12559791e-02,
        3.14569995e-02,  1.93744805e-02,  4.65708449e-02,  7.75612593e-02,
       -6.96181972e-03,  4.17182781e-02, -1.53666250e-02, -5.87033890e-02,
       -3.97401042e-02,  2.02280004e-03, -3.72142391e-03,  1.81982783e-03,
       -6.34902203e-03, -1.87578723e-02,  1.01454407e-02, -2.06669103e-02,
        6.53604642e-02, -1.16215926e-02,  2.09548250e-02,  1.80853400e-02,
        4.71785143e-02, -4.51582596e-02, -3.30339372e-02, -7.94724524e-02,
        3.49714309e-02,  2.18167771e-02,  2.13386238e-06, -2.68752221e-02,
        3.07219606e-02, -5.75455837e-03, -1.94926392e-02, -9.64277983e-03,
        4.39838283e-02,  7.33801723e-02, -2.97792256e-02, -3.74800600e-02,
       -1.49240866e-02,  7.23522948e-03,  1.71101484e-02, -3.71705741e-02,
        3.25852856e-02,  6.93616495e-02, -1.82072297e-02,  4.11012694e-02,
       -7.68169239e-02, -

## Define User Request

We will define a user request. This is a question that we would like to ask the model. We will calculate the embedding for this question and use it to retrieve the most relevant articles from the travel articles by comparing the vector of the User Request to the other vectors we just calculated above.

<img src="images/2_define_UR.png" alt="Image description" width="200">  


In [12]:
# Provide the User Request as a string, e.g., user_request = "I want to travel to a Greek Island that is famous this season for snorkeling."
user_request = "I want to travel to a Greek Island that is famous this season for snorkeling. Which one would you recommend to me?"

## Calculate the embeddings for the User Request

In the same fashion as before, we calculate the embeddings for the User Request using the same sentence transformer model that we used before - `all-mpnet-base-v2`.

The vector will have the same shape as the vectors we calculated for the travel articles. Had we used a different model, the vectors would likely have been of a different size. In any event, this would have made it impossible to compare the vectors, we would compare apples to oranges.

<img src="images/3_calculate_embeddings.png" alt="Image description" width="200">  


In [13]:
user_request_embedding = embedding_model.encode(user_request)

print(f"Shape of the user request vector: {user_request_embedding.shape}")

Shape of the user request vector: (768,)


## Select the N most similar articles

We will compare the User Request vector to the vectors of the travel articles. There are different ways to compare vectors. For simplicity, we will use the cosine similarity.

In principle, we could select any number N of articles. To honor token limits (globally for the endpoint we are using and locally in each request), we will select the top 3 articles.

<img src="images/4_select_n.png" alt="Image description" width="200">  


In [14]:
def distance_between_vector_and_vectors(
    vector: np.ndarray, vectors_array: np.ndarray
    ) -> np.ndarray:
    """
    Calculate the cosine similarities between a single vector and an array of vectors.

    Args:
        vector: A single vector
        vectors_array: An array of vectors
    
    Returns:
        An array of cosine similarities
    """
    dot_products = np.dot(vectors_array, vector)

    # Calculate the magnitudes of all vectors in vectors_array
    magnitudes = np.sqrt(np.sum(np.square(vectors_array), axis=1))

    # Calculate the magnitude of vector
    magnitude_1 = np.linalg.norm(vector)

    # Calculate the cosine similarities between vector and all vectors in vectors_array
    similarities = dot_products / (magnitude_1 * magnitudes)

    return similarities


In [15]:
k = 3 # You can increase this number, but you might run into a token limit

distances = distance_between_vector_and_vectors(
    vector=user_request_embedding,
    vectors_array=embeddings
)

idx_of_most_similar = list((-distances).argsort()[:k])

print(f"Most similar articles are articles numbers: {idx_of_most_similar} with similarities: {distances[idx_of_most_similar]}")

Most similar articles are articles numbers: [9, 13, 36] with similarities: [0.70965517 0.6991749  0.6891067 ]


## Retrieve the text corresponding to the most similar articles embeddings

We will retrieve the text of the most similar articles. We do this by looking up the index of the most similar articles in the list of travel articles.

Check if the articles make sense and are relevant to the user request that you defined.

<img src="images/5_retrieve_text.png" alt="Image description" width="200">  


In [16]:
most_similar_articles = [travel_articles[i] for i in idx_of_most_similar]

for i, article in enumerate(most_similar_articles):
    print(f"Article {i+1}")
    display(Markdown(article[:200]))

Article 1


Title: Dive into the Mesmerizing Underwater World of Syros: The Ultimate Snorkeling Paradise

Introduction

Golden beaches, crystal-clear waters, and a rich cultural heritage – welcome to Syros, the c

Article 2


Title: Discover the Underwater Wonderland of Kythnos: The World's Best Snorkeling Destination

Introduction

Nestled in the heart of the Cyclades, the enchanting island of Kythnos is one of Greece's b

Article 3


Title: Crete: Dive into the Ultimate Ocean Swimming Paradise

Introduction

The island of Crete, the largest and most populous of the Greek islands, has long been renowned for its crystal-clear waters

## Augment the Prompt with the retrieved text

We will augment the prompt with the retrieved text. This will provide the model with the private domain knowledge it would otherwise not know about and that it can use to generate a more accurate response.

<img src="images/6_augment_prompt.png" alt="Image description" width="200">  


### Set the character (system message) below as a string. 

Set the character (system message) below as a string. 

When using the OpenAI API, both setting a system message and including the character in the prompt will serve to guide the model's behavior.
A system message is often used with OpenAI's Chat API, where the conversation occurs in a more interactive and dynamic manner. By using a system message, you can provide instructions or context to the AI model. This message isn't counted towards the token limit. However, it may have less influence on the response compared to including the character directly in the prompt.

Some example system messages are:
- You are a marketing writing assistant. You help come up with creative content ideas and content like marketing emails, blog posts, tweets, ad copy and product descriptions. You write in a friendly yet professional tone but can tailor your writing style that best works for a user-specified audience. If you do not know the answer to a question, respond by saying "I do not know the answer to your question."
- Assistant is an AI chatbot that helps users turn a natural language list into JSON format. After users input a list they want in JSON format, it will provide suggested list of attribute labels if the user has not provided any, then ask the user to confirm them before creating the list.
- You are an Xbox customer support agent whose primary goal is to help users with issues they are experiencing with their Xbox devices. You are friendly and concise. You only provide factual answers to queries, and do not provide answers that are not related to Xbox.

In [17]:
# Set the character
character = "You are a seasoned travel agent with the primary goal to help users looking to plan a vacation in Greece. You write in a friendly yet professional tone."

Next, we have to build the question to ask the model enriched with the information from the embeddings.

Below you find an example how the augmented prompt could look like (using the `user request` that you defined). You can use this as a template to build your own question if you want.

In [18]:
context_instruction = 'Use the below travel articles about Greek Islands from the current travel season to answer \
the subsequent question. If the answer cannot be found in the articles, write "I could \
not find an answer."'
augmented_prompt = f"{context_instruction} \n\n Question: {user_request} \n\n The travel articles follow below: \n\n"

for piece in most_similar_articles:
    augmented_prompt += piece

# Uncomment the below line to see the result of the example
display(Markdown(augmented_prompt))

Use the below travel articles about Greek Islands from the current travel season to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer." 

 Question: I want to travel to a Greek Island that is famous this season for snorkeling. Which one would you recommend to me? 

 The travel articles follow below: 

Title: Dive into the Mesmerizing Underwater World of Syros: The Ultimate Snorkeling Paradise

Introduction

Golden beaches, crystal-clear waters, and a rich cultural heritage – welcome to Syros, the crown jewel of the Cyclades Islands in Greece. With its neoclassical mansions, labyrinthine streets, and captivating history, Syros has long been a favorite destination for discerning travelers. But what truly sets this enchanting island apart is its unparalleled snorkeling opportunities. In fact, Syros has earned a well-deserved reputation as the world's best snorkeling destination. So pack your bags, grab your snorkel gear, and join us on a journey to explore the underwater wonders that await you in this sun-soaked paradise.

The Perfect Snorkeling Destination

The pristine waters surrounding Syros boast an astonishing array of marine life, coral formations, and underwater caves that can't be found anywhere else in the world. The island's unique geological features have created a diverse underwater ecosystem, teeming with vibrant fish, colorful corals, and mysterious sea creatures that will leave even the most seasoned snorkeler in awe. The clarity of the water – with visibility of up to 50 meters – ensures that every snorkeling adventure is an unforgettable experience.

A Snorkeler's Dream: The Breathtaking Dive Sites of Syros

With dozens of snorkeling sites scattered around the island, Syros has something for everyone – from beginners to seasoned pros. Here are just a few of the must-visit spots that have made Syros famous among snorkeling enthusiasts worldwide:

1. The Coral Gardens: Located just off the coast of Syros, this shallow reef is brimming with a wide variety of coral species and an abundance of marine life. Expect to encounter parrotfish, angelfish, damselfish, and countless other colorful species as you glide through this mesmerizing underwater garden.

2. The Shipwreck of Agios Nikolaos: Dive into history as you explore the remains of this 19th-century shipwreck, which has become an artificial reef teeming with marine life. The wreck sits at a depth of 12 meters and is home to schools of fish, octopuses, and even the occasional sea turtle.

3. The Caverns of Kini: This intricate network of underwater caves and tunnels is a playground for experienced snorkelers. Navigate through the labyrinthine passageways, marveling at the stunning coral formations and the ethereal shafts of sunlight that pierce the darkness.

4. The Dolphin's Playground: This aptly-named site is a favorite hangout for a pod of friendly dolphins that love to interact with snorkelers. With a little luck, you'll find yourself swimming alongside these playful creatures, creating memories that will last a lifetime.

5. The Seahorse Sanctuary: Tucked away in a secluded cove, this magical spot is home to a thriving population of seahorses – one of the ocean's most enchanting and elusive creatures. Watch in wonder as these graceful beings dance and twirl through the water, their delicate features and intricate markings on full display.

Snorkeling Tours and Courses: Enhance Your Experience

Whether you're a first-time snorkeler or a seasoned pro, Syros offers a range of guided tours and courses to help you make the most of your underwater adventures. Expert guides will take you to the island's best snorkeling spots, sharing their knowledge of the local marine life and ensuring your safety at all times. For those looking to improve their skills, a variety of PADI-certified courses are available, from beginner lessons to advanced freediving techniques.

Family-Friendly Fun: Snorkeling for All Ages

Snorkeling is an activity that can be enjoyed by the entire family, and Syros is the perfect destination for introducing your little ones to the wonders of the underwater world. With its calm, shallow waters and abundance of marine life, the island offers a safe and exciting environment for children to learn about the ocean and its inhabitants. Many of the island's snorkeling operators offer special programs and equipment tailored for young explorers, ensuring that your family vacation is filled with unforgettable experiences.

Beyond Snorkeling: Discover the Rich Culture of Syros

When you're not exploring the island's underwater treasures, Syros offers a wealth of cultural experiences to enjoy. Stroll through the charming streets of Ermoupoli, the island's capital, and marvel at the elegant neoclassical architecture that harks back to a bygone era. Visit the impressive Apollo Theatre, a miniature replica of Milan's famous La Scala opera house, and catch a performance by local artists. For a taste of the island's rich culinary heritage, indulge in traditional Greek dishes at one of the many tavernas, where you'll be greeted with warm hospitality and mouthwatering flavors.

With its breathtaking underwater landscapes, unique marine life, and rich cultural offerings, Syros is truly a destination like no other. Whether you're a seasoned snorkeler or just starting out, this enchanting island is sure to capture your heart and leave you longing for more. So why wait? Dive into the adventure of a lifetime and discover the unparalleled snorkeling paradise that is Syros.Title: Discover the Underwater Wonderland of Kythnos: The World's Best Snorkeling Destination

Introduction

Nestled in the heart of the Cyclades, the enchanting island of Kythnos is one of Greece's best-kept secrets. Known for its pristine beaches, charming villages, and ancient heritage, Kythnos offers the perfect blend of relaxation and adventure. But what truly sets this island apart from other destinations is its world-renowned snorkeling opportunities. Boasting crystal-clear waters teeming with vibrant marine life, Kythnos is the ultimate paradise for snorkeling enthusiasts of all levels. In this article, we'll guide you through the wonders of Kythnos and reveal why this island should top your travel bucket list.

The Island of Kythnos: An Overview

Located between Kea and Serifos, Kythnos forms part of the Western Cyclades group of islands, a region famous for its idyllic landscapes and rich history. With its stunning coastline stretching over 100 kilometers, Kythnos is home to more than 70 beaches, many of which are only accessible by boat or foot.

The island is characterized by its traditional Cycladic architecture, featuring white-washed houses with blue windows and doors, narrow alleys, and picturesque windmills. Kythnos is also known for its hot springs, particularly the ones at Loutra, which have been attracting visitors for centuries due to their therapeutic properties.

But it's the island's underwater world that truly steals the spotlight. Thanks to its unique topography, Kythnos boasts a diverse marine ecosystem, making it a snorkeling paradise. From mesmerizing coral reefs to mysterious shipwrecks, Kythnos offers unparalleled snorkeling experiences that you won't find anywhere else in the world.

World-Class Snorkeling: What Makes Kythnos Stand Out

With its crystal-clear waters, warm temperatures, and extraordinary biodiversity, Kythnos is the best place in the world for snorkeling enthusiasts. Here's what makes this island truly unique:

1. Exceptional Visibility: Kythnos' waters are renowned for their exceptional visibility, which can reach up to 50 meters on a good day. This allows snorkelers to fully immerse themselves in the underwater world, making it an ideal destination for underwater photography.

2. Diverse Marine Life: Kythnos' marine ecosystem is home to an incredible variety of species, including colorful fish, vibrant corals, and enchanting sea creatures. Some of the species you can encounter while snorkeling in Kythnos include parrotfish, damselfish, wrasses, octopuses, sea urchins, and even the occasional sea turtle.

3. Unique Underwater Landscapes: The island's underwater topography features a fascinating combination of coral gardens, underwater caves, and ancient shipwrecks. These diverse landscapes provide a thrilling snorkeling experience, as you never know what you might discover around the next corner.

4. Easily Accessible: Unlike other world-class snorkeling destinations, Kythnos is easily accessible by ferry from Athens, making it a convenient choice for travelers looking to combine their snorkeling adventure with a visit to the Greek capital.

5. Suitable for All Levels: Whether you're a seasoned snorkeler or a complete beginner, Kythnos offers snorkeling spots suitable for all levels. There are several professional diving centers on the island, offering guided snorkeling tours and equipment rental for those who need it.

Top Snorkeling Spots in Kythnos

While the entire coastline of Kythnos offers incredible snorkeling opportunities, there are a few spots that stand out for their exceptional beauty and diversity:

1. Kolona Beach: Considered the crown jewel of Kythnos, this stunning double-sided beach is a snorkeler's paradise. The calm, shallow waters are ideal for beginners, while the deeper areas near the rocky coastline are teeming with marine life and colorful corals, perfect for more experienced snorkelers.

2. Apokrousi Beach: Located on the western side of the island, Apokrousi beach offers a unique snorkeling experience. Here, you can explore an underwater cave with a mesmerizing array of stalactites and stalagmites, as well as a vibrant coral reef populated by a wide variety of fish species.

3. Loutra Bay: The crystal-clear waters of Loutra Bay are home to an ancient shipwreck, which now serves as an artificial reef attracting an abundance of marine life. Snorkelers can explore the remains of the ship, which is teeming with colorful fish, sea sponges, and other fascinating creatures.

4. Naoussa Beach: This secluded beach is accessible only by boat, making it the perfect spot for those seeking a more remote snorkeling experience. The rocky coastline provides an ideal habitat for a diverse range of marine species, including octopuses, sea urchins, and a variety of colorful fish.

5. Agios Sostis Beach: With its shallow, turquoise waters and sandy seabed, Agios Sostis beach is an excellent choice for families and beginner snorkelers. The calm bay is home to a diverse range of marine life, including sea turtles, which are known to frequent the area.

Beyond Snorkeling: Other Activities to Enjoy in Kythnos

While snorkeling may be the main draw of Kythnos, the island offers plenty of other activities for visitors to enjoy:

1. Hiking: With its rolling hills and scenic trails, Kythnos is a hiker's dream. Explore the island's diverse landscapes, including the lush green valleys of Dryopida, the rugged cliffs of Loutra, and the picturesque windmills of Chora.

2. Hot Springs: No visit to Kythnos would be complete without a dip in the island's famous hot springs. The therapeutic waters of Loutra are said to have healing properties, making them the perfect way to unwind after a day of snorkeling.

3. Village-Hopping: Discover the island's rich history and culture by visiting its traditional villages. Wander the narrow alleys of Chora, admire the Venetian castle in Driopida, and watch the sunset from the picturesque village of Kanala.

4. Gastronomy: Indulge in Kythnos' mouthwatering cuisine, which features a range of delicious dishes made with fresh, locally-sourced ingredients. Don't miss the chance to sample the island's famous cheese, known as sfela, or its traditional almond cookies, called amygdalota.

Conclusion

If you're seeking an unforgettable snorkeling adventure, look no further than the island of Kythnos. With its unparalleled underwater landscapes, diverse marine life, and exceptional visibility, Kythnos has earned its reputation as the world's best snorkeling destination. So pack your snorkel gear, book your tickets, and prepare to embark on a once-in-a-lifetime journey to this enchanting Greek island.Title: Crete: Dive into the Ultimate Ocean Swimming Paradise

Introduction

The island of Crete, the largest and most populous of the Greek islands, has long been renowned for its crystal-clear waters, sun-soaked shores, and breathtaking landscapes. But what truly sets Crete apart from other travel destinations is its unparalleled ocean swimming experience. With its stunning underwater world, tranquil bays, and pristine beaches, Crete is undeniably the best place in the world for swimming in the ocean. In this article, we will dive deep into the heart of Crete and explore the island's mesmerizing aquatic wonders, irresistible charm, and the many activities one can indulge in on this enchanting island.

The Allure of Crete's Waters

Crete's coastline stretches over 1,046 kilometers, offering an abundance of picturesque swimming spots and hidden gems just waiting to be discovered. The island's location in the Mediterranean Sea, combined with its mild climate, makes it an ideal destination for swimming enthusiasts year-round. The waters surrounding Crete are a vibrant turquoise, providing excellent visibility for snorkelers and divers alike, and the pleasant water temperature ranging from 20°C to 27°C during the summer months is perfect for extended ocean swims.

The island's diverse coastline offers swimmers a wide variety of swimming experiences to choose from, whether it's exploring secluded coves or taking a dip in the gentle waves of a sandy beach. And with over 100 beaches awarded the prestigious Blue Flag – a certification recognizing the high environmental and quality standards – Crete promises an unmatched ocean swimming experience that will leave you longing for more.

Swimming Hotspots

While Crete boasts countless idyllic swimming locations, there are a few standouts that should be on every ocean swimmer's bucket list. These pristine spots offer not only the best swimming conditions but also showcase Crete's natural beauty and rich history.

1. Elafonisi Beach: Often ranked as one of the world's best beaches, Elafonisi is a must-visit for any ocean swimmer. Its powdery pink sand and shallow, warm waters create a dreamlike setting for a leisurely swim. The nearby Elafonisi Island, accessible by a sandbar, is home to a protected nature reserve, offering a serene escape from the main beach.

2. Balos Lagoon: This breathtaking lagoon is a true swimming paradise. The crystal-clear turquoise waters are calm and shallow, providing a safe environment for swimmers of all levels. With its white sand and exotic surroundings, Balos Lagoon feels like a slice of the Caribbean in the heart of the Mediterranean.

3. Preveli Beach: Nestled at the mouth of the Kourtaliotiko Gorge, Preveli Beach is a unique swimming destination. The beach is famous for its lush palm forest and river, which flows into the sea, creating a refreshing freshwater swimming spot. The ocean waters are calm and clear, perfect for exploring the area's abundant marine life.

4. Vai Beach: Known for its iconic palm forest – the largest in Europe – Vai Beach offers swimmers a truly exotic experience. The golden sand and crystal-clear waters create a stunning contrast, and the calm sea conditions make it an ideal location for a relaxing swim.

5. Matala Beach: Steeped in history, Matala Beach was once a popular hippie destination in the 1960s and '70s. Today, the beach's golden sand and azure waters attract swimmers from all over the world. The nearby caves, carved into the cliffside, add an element of intrigue to this picturesque swimming spot.

Aquatic Activities

Crete's diverse aquatic landscape offers a range of water-based activities for visitors to enjoy. From snorkeling and scuba diving to sailing and windsurfing, there's something for everyone on this enchanting island.

1. Snorkeling and Scuba Diving: Crete's crystal-clear waters and rich marine life make it a haven for underwater exploration. Popular diving spots include the ancient shipwreck at Dia Island and the vibrant reefs of Chania's Agioi Apostoloi.

2. Windsurfing and Kitesurfing: With its steady winds and warm waters, Crete is a popular destination for windsurfing and kitesurfing enthusiasts. Top locations for these thrilling water sports include Elafonisi, Falassarna, and Palekastro.

3. Sailing: The best way to experience Crete's captivating coastline is by boat. Charter a yacht or join a sailing tour to explore hidden coves, remote beaches, and unspoiled islands.

4. Canoeing and Kayaking: For a more serene water experience, rent a canoe or kayak and paddle through the calm waters of Crete's rivers, lakes, and sea. Popular spots include the tranquil waters of Lake Kournas and the imposing Kourtaliotiko Gorge.

5. Fishing: Try your hand at traditional fishing techniques with a local fisherman, or join a deep-sea fishing excursion for a chance to catch a prized Mediterranean fish.

Onshore Activities

While Crete's aquatic wonders are undoubtedly the main attraction, the island offers a plethora of onshore activities for visitors to enjoy. From hiking and cycling to exploring ancient ruins and indulging in mouthwatering cuisine, Crete has it all.

1. Hiking: Crete's diverse landscape offers a multitude of hiking trails for all levels of fitness. Popular routes include the stunning Samaria Gorge, the lush Imbros Gorge, and the dramatic coastal path of the E4 European Long Distance Path.

2. Cycling: Rent a bike and explore Crete's picturesque countryside, quaint villages, and beautiful coastline at your own pace.

3. Historical Sites: The island's rich history is evident in its numerous archaeological sites, including the ancient Minoan palace of Knossos and the Venetian harbor of Chania.

4. Culinary Delights: Crete's cuisine is a delicious blend of local ingredients and Mediterranean flavors. Be sure to sample the island's signature dishes, such as dakos, kalitsounia, and of course, the famous Cretan olive oil.

5. Wellness Retreats: Unwind and rejuvenate at one of Crete's many wellness retreats, offering yoga, meditation, and spa treatments in serene and beautiful surroundings.

Conclusion

Crete is undeniably the ultimate ocean swimming destination, offering a perfect blend of pristine waters, stunning natural beauty, and a rich cultural heritage. With its diverse range of activities both in and out of the water, Crete promises an unforgettable travel experience that will leave you yearning to return to its enchanting shores. So pack your swimsuit, dive into the crystal-clear waters of Crete, and discover the unmatched aquatic wonders of this Mediterranean paradise.

## Send the augmented prompt to the LLM

We will send the augmented prompt to the LLM and get a response back. The response will be the completion generated by the model.

<img src="images/7_to_llm.png" alt="Image description" width="200">  


### Build the prompt 

The OpenAI API expects the "prompt" to be in a specific form:
> prompt = [ \
>    {"role": "system", "content": character goes here}, \
>    {"role": "user", "content": the question you wish to ask goes here}, \
>] 

We build the prompt below referring to the system message and augmented prompt that you defined.

In [19]:
prompt = [
    {
        "role": "system",
        "content": character
    },
    {
        "role": "user",
        "content": augmented_prompt
    },
]

In [20]:
def ask_question(
        prompt: List[Dict[str, str]],
        model: str = MODEL
    ) -> openai.types.chat.chat_completion.ChatCompletion:
    """Function to ask a question to the GPT model using the Azure OpenAI API.
    
    Args:
        prompt: The prompt to send to the GPT model
        model: The model to use

    Returns:
        The response from the GPT model
    """

    response = client.chat.completions.create(
        model=model,
        messages=prompt,
        temperature=0.7,
        max_tokens=1500,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
    )
    
    return response

def get_cost(
        response: openai.types.chat.chat_completion.ChatCompletion,
        token_cost_per_completion: float,
        token_cost_per_prompt: float,
    ) -> float:
    """Function to compute the cost of a prompt + completion response.
    
    Args:
        response: The response from the GPT model
        token_cost_per_completion: The cost per completion token
        token_cost_per_prompt: The cost per prompt token
    
    Returns:
        The cost of the prompt + completion response
    """
    completion_tokens = response.usage.completion_tokens
    prompt_tokens = response.usage.prompt_tokens

    cost = (completion_tokens * token_cost_per_completion) + (
        prompt_tokens * token_cost_per_prompt
    )

    return cost


In [21]:
# Use `ask_question` to use the Azure OpenAI API to interact with the model

response = ask_question(
    prompt=prompt,
    model=MODEL
)

# Consider also the cost
cost = get_cost(
    response=response,
    token_cost_per_completion=model_cost.token_cost_per_completion,
    token_cost_per_prompt=model_cost.token_cost_per_prompt
)

print(f"Total cost for this response: {cost:.5f} {model_cost.currency} (model was {model_cost.name}).")

Total cost for this response: 0.00581 CHF (model was gpt3.5-turbo).


In [22]:
# Display the answer from the model nicely formatted
answer = response.choices[0].message.content

display(Markdown(answer))

Based on the travel articles, the Greek Island of Syros is the famous snorkeling destination this season. Its pristine waters boast an astonishing array of marine life, coral formations, and underwater caves that can't be found anywhere else in the world. The island's unique geological features have created a diverse underwater ecosystem, teeming with vibrant fish, colorful corals, and mysterious sea creatures. The clarity of the water ensures that every snorkeling adventure is an unforgettable experience. With dozens of snorkeling sites scattered around the island, Syros has something for everyone – from beginners to seasoned pros, making it the ultimate snorkeling paradise.

# Congratulations!

You augmented the prompt with the retrieved text from the travel articles. You can use this approach to retrieve information from any text collection that you have. You can also use this approach to retrieve information from a collection of documents that you have in your company. For example, you could use this approach to retrieve information from your company's internal wiki or your own E-Mail inbox (if you're aware that you will be sending all the data to the model).

However, as usual, be aware of the privacy and security implications of sending data to the model - Think before you hit send.

If you want to further explore and try out different questions to ask, you can use the methods that we provided for you below in the "Talk to GPT" section.

# Talk to GPT

Using the steps above, you can use below cell to play around with the OpenAI API and the travel articles we provided. You can ask the model any question you like.

In the cell directly below we define a few helper functions to make your life easier. You can use them to send a prompt to GPT and display the result. The methods are the same as above, you have to copy your work down, for example for the system message and the augmented prompt.

You will need to have loaded the travel articles and calculated the embeddings for them in order to use the methods below.

Try to test the limits, e.g., asking about an activity like "walking on the moon", or asking about a location that is not in the travel articles.

In [23]:
def distance_between_vector_and_vectors(
    vector: np.ndarray, vectors_array: np.ndarray
    ) -> np.ndarray:
    """
    Calculate the cosine similarities between a single vector and an array of vectors.

    Args:
        vector: A single vector
        vectors_array: An array of vectors
    
    Returns:
        An array of cosine similarities
    """
    dot_products = np.dot(vectors_array, vector)

    # Calculate the magnitudes of all vectors in vectors_array
    magnitudes = np.sqrt(np.sum(np.square(vectors_array), axis=1))

    # Calculate the magnitude of vector
    magnitude_1 = np.linalg.norm(vector)

    # Calculate the cosine similarities between vector and all vectors in vectors_array
    similarities = dot_products / (magnitude_1 * magnitudes)

    return similarities


def get_pieces_of_interest(question: str, k: int = 3) -> List[str]:
    """Function to get the pieces of interest for a question.
    
    Args:
        question: The question to ask the model
        k: The number of pieces of interest to find
    
    Returns:
        The pieces of interest
    """
    question_embedding = embedding_model.encode(question, show_progress_bar=True)

    distances = distance_between_vector_and_vectors(
        vector=question_embedding,
        vectors_array=embeddings
    )

    idx_of_interest = list((-distances).argsort()[:k])
    pieces_of_interest = [travel_articles[i] for i in idx_of_interest]

    return pieces_of_interest


def get_augmented_prompt(
        question: str,
        pieces_of_interest: List[str],
        context_instruction: str,
    ) -> str:
    """Function to build the augmented prompt.
    
    Args:
        question: The question to ask the model
        pieces_of_interest: The pieces of interest
        context_instruction: The context instruction
    
    Returns:
        The augmented prompt
    """
    augmented_prompt = f"{context_instruction} \n\n Question: {question} \n\n The travel articles follow below: \n\n"

    for piece in pieces_of_interest:
        augmented_prompt += piece

    return augmented_prompt

# Define a user request
user_request = "I want to travel to a Greek Island that is famous this season for eating good food. Which one would you recommend to me?"

# Select k, the number of pieces of interest
k = 3

pieces_of_interest = get_pieces_of_interest(question=user_request, k=k)

# Set the character
character = "You are a seasoned travel agent with the primary goal to help users looking to plan a vacation in Greece. You write in a friendly yet professional tone."

# Provide your augmented prompt
context_instruction = 'Use the below travel articles about Greek Islands from the current travel season to answer \
the subsequent question. If the answer cannot be found in the articles, write "I could \
not find an answer."'
augmented_prompt = get_augmented_prompt(
        question=user_request,
        pieces_of_interest=pieces_of_interest,
        context_instruction=context_instruction,
)

# Build the prompt
prompt = [
    {
        "role": "system",
        "content": character
    },
    {
        "role": "user",
        "content": augmented_prompt
    },
]

# Send the prompt to the model
response = ask_question(
    prompt=prompt,
    model=MODEL
)

# Consider the cost
cost = get_cost(
    response=response,
    token_cost_per_completion=model_cost.token_cost_per_completion,
    token_cost_per_prompt=model_cost.token_cost_per_prompt
)
print(f"Total cost for this response: {cost:.5f} {model_cost.currency} (model was {model_cost.name}).")


# Display the answer from the model nicely formatted
answer = response.choices[0].message.content

display(Markdown(answer))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Total cost for this response: 0.00573 CHF (model was gpt3.5-turbo).


Based on the travel articles, Thassos is the Greek Island that is famous this season for its good food. Thassos is described as a gastronomic treasure trove and the best place in the world for fine dining. The island boasts an impressive array of Michelin-starred restaurants, innovative eateries, and vibrant food markets that will satisfy even the most discerning palate. The island's unique microclimate and fertile soil allow for the production of some of the finest ingredients on the planet, and the culinary creativity of Thassos' talented chefs makes it a truly out-of-this-world dining experience. Therefore, Thassos is the recommended Greek Island for its exceptional gastronomic offerings this season.